# Lesson  3 Building the Tutor Service Layer

Absolutely! Here’s your text, **converted to markdown** with headings, code blocks, and clean formatting for readability:

---

# Building the Tutor Service Layer

In the previous lesson, we explored the `SessionManager` class, which plays a crucial role in managing tutoring session data within our application. Now, we will take the next step in our journey by building the **Tutor Service Layer**. This layer is essential for integrating the DeepSeek language model with tutoring sessions, allowing us to process student queries and generate tailored explanations. By the end of this lesson, you will understand how to set up the `TutorService` class, create tutoring sessions, and process academic questions using DeepSeek models via the OpenAI SDK.

The service layer acts as a bridge between the model layer, where data is managed, and the AI model, which generates educational responses. It is responsible for orchestrating the flow of data and ensuring that student interactions are handled effectively. Let's dive into the details of setting up this important component.

---

## Setting Up the TutorService Class

The `TutorService` class is the heart of our service layer. It is responsible for managing tutoring sessions and interacting with the DeepSeek model to generate educational responses. To begin, we need to set up the class and its components.

First, we import the necessary modules, including the `SessionManager` from our previous lesson and the OpenAI client (which we'll use to access DeepSeek models). We also use the `uuid` module to generate unique session IDs. Here’s how the class is initialized:

```python
import uuid
from openai import OpenAI
from models.session import SessionManager

class TutorService:
    def __init__(self):
        self.session_manager = SessionManager()
        self.deepseek_client = OpenAI()
        self.system_prompt = self.load_system_prompt('data/system_prompt.txt')
```

In this setup, we instantiate `SessionManager` to manage tutoring data, initialize the OpenAI client for DeepSeek model access, and load the `system_prompt` using the `load_system_prompt` method, which we'll discuss next.

---

## Loading the System Prompt

The system prompt is a crucial component that guides the tutor AI's responses. It provides context and instructions for the AI, ensuring that it behaves like an effective educational assistant. In this section, we'll implement the `load_system_prompt` method to load the prompt from a file.

```python
def load_system_prompt(self, file_path: str) -> str:
    """Load the tutor system prompt from file."""
    try:
        with open(file_path, 'r') as f:
            return f.read()
    except Exception as e:
        print(f"Error loading system prompt: {e}")
        return "You are a helpful tutor."
```

This method attempts to read the system prompt from a specified file path. If successful, it returns the prompt as a string. In case of an error, it prints an error message and returns a default prompt. This ensures that the application can continue functioning even if the file is missing or corrupted.

---

## Creating a New Tutoring Session

Creating a new tutoring session is a fundamental task of the `TutorService`. The `create_session` method is responsible for generating a unique session ID and initializing a tutoring session using the `SessionManager`.

```python
def create_session(self, student_id: str) -> str:
    """Create a new tutoring session."""
    session_id = str(uuid.uuid4())
    self.session_manager.create_session(student_id, session_id, self.system_prompt)
    return session_id
```

In this method, we generate a unique `session_id` using the `uuid` module. We then call the `create_session` method of `SessionManager`, passing the `student_id`, `session_id`, and `system_prompt`. This initializes a new tutoring session, which is ready to receive student queries.

---

## Processing Student Queries

The `process_query` method is where the educational magic happens. It processes student questions, interacts with the DeepSeek model to generate tutoring explanations, and updates the session history. Below, we outline the steps involved in this process, followed by the corresponding code implementation:

* Retrieve the session using `get_session`, and raise an error if the session is not found.
* Add the student's query to the session history.
* Send the conversation, including the system prompt and all previous exchanges, to the DeepSeek model to generate a response.
* Add the tutor's explanation to the session history and return it to the student.
* Handle any errors with the AI client gracefully.

```python
def process_query(self, student_id: str, session_id: str, query: str) -> str:
    """Process a student query and provide a tutoring explanation."""
    
    # Step 1: Retrieve the session
    session = self.session_manager.get_session(student_id, session_id)
    if not session:
        raise ValueError("Session not found")
    
    # Step 2: Add student's query to session
    self.session_manager.add_message(student_id, session_id, "user", query)
    
    try:
        # Step 3: Get tutor response
        conversation = self.session_manager.get_conversation(student_id, session_id)
        
        response = self.deepseek_client.chat.completions.create(
            model="deepseek-ai/DeepSeek-V3",
            messages=conversation,
            temperature=0.6,
            max_tokens=500
        )
        
        tutor_response = response.choices[0].message.content.strip()
        
        # Step 4: Add tutor's explanation to session history
        self.session_manager.add_message(student_id, session_id, "assistant", tutor_response)
        
        return tutor_response
        
    except Exception as e:
        # Step 5: Handle errors
        raise RuntimeError(f"Error getting tutor response: {str(e)}")
```

In the context of a personal tutor, we configure our DeepSeek model with specific parameters to optimize its educational performance. The temperature is set to `0.6`, which balances accuracy and creativity in the tutor's explanations, ensuring they are both informative and engaging. The `max_tokens` is set to `500`, allowing the model to provide detailed educational content without overwhelming the student, thus maintaining an effective learning experience.

---

## Example: Simulating a Tutoring Session

Let's see the `TutorService` in action by simulating a tutoring session. We'll create a script to initialize a tutoring session and process a student's academic query.

```python
from services.tutor_service import TutorService

# Initialize the tutor service
tutor_service = TutorService()

# Simulate a student ID
student_id = "student123"

# Create a new tutoring session
session_id = tutor_service.create_session(student_id)
print(f"Tutoring session created with ID: {session_id}")

# Simulate sending a tutoring query
student_query = "Can you explain the principles of supply and demand in economics?"

try:
    tutor_response = tutor_service.process_query(student_id, session_id, student_query)
    print(f"Tutor Response: {tutor_response}")
except Exception as e:
    print(f"Error: {e}")
```

**Plain text output:**

```
Tutoring session created with ID: 01a17870-8a4f-4b6f-a3ce-f04e1136d597
Tutor Response: Supply and demand are fundamental principles in economics that describe how prices are determined in a market economy. Let me explain each concept and how they interact:

1. Supply: This refers to the quantity of a good or service that producers are willing and able to offer for sale at various price points. 
   - The law of supply states that as the price of a good increases, the quantity supplied also increases (and vice versa).
   - This creates an upward-sloping supply curve when plotted on a graph with price on the vertical axis and quantity on the horizontal axis.

2. Demand: This refers to the quantity of a good or service that consumers are willing and able to purchase at various price points.
   - The law of demand states that as the price of a good increases, the quantity demanded decreases (and vice versa).
   - This creates a downward-sloping demand curve on the same type of graph.

3. Market Equilibrium: This occurs at the intersection of the supply and demand curves.
   - At this point, the quantity that producers want to supply exactly equals the quantity that consumers want to buy.
   - The price at this intersection is called the equilibrium price, and the quantity is called the equilibrium quantity.

4. Price Mechanism: When markets are not in equilibrium, the price acts as a signal:
   - If price is above equilibrium, there's a surplus (excess supply), which puts downward pressure on prices.
   - If price is below equilibrium, there's a shortage (excess demand), which puts upward pressure on prices.

5. Shifts in Supply and Demand: Various factors can cause entire curves to shift:
   - Supply shifters include technology, input costs, number of sellers, and expectations.
   - Demand shifters include income, preferences, number of buyers, and expectations.

These principles help explain how markets allocate resources efficiently and how changes in market conditions affect prices and quantities.

Does this explanation help? Would you like me to elaborate on any specific aspect of supply and demand?
```

This output illustrates a successful tutoring interaction where a new session is created, and the AI responds to the student's economics question with a comprehensive explanation. The tutor's response demonstrates the system's ability to provide relevant, structured, and educational content, showcasing how the DeepSeek model can be effectively used for personalized academic support.

---

## Summary and Next Steps

In this lesson, we explored the `TutorService` class and its role in integrating the DeepSeek language model with tutoring sessions. We learned how to set up the class, load the system prompt, create sessions, and process student queries. The service layer is a vital component of our personal tutor application, ensuring that student interactions are handled effectively and that educational content is delivered in a clear and engaging manner.

As you move on to the practice exercises, take the opportunity to experiment with the `TutorService` functionality. This hands-on practice will reinforce the concepts covered in this lesson and prepare you for the next steps in our course. Keep up the great work, and I look forward to seeing your progress!

---


## Building the Tutor Service Layer

Implement the create_session method within the TutorService class.

Use the uuid module to generate a unique session ID.
Utilize the SessionManager to create a tutoring session, associating it with the student ID, session ID, and system prompt.
Ensure the method returns the generated session ID.
Test the session creation using the service.

Initialize an instance of TutorService.
Define a sample student ID to simulate a student.
Call the create_session method with the student ID and store the returned session ID.
Print the session ID to verify the successful creation of the tutoring session.
This task will guide you through the process of creating and managing tutoring sessions, a fundamental aspect of building a personal tutor service.

```python
import uuid
from models.session import SessionManager


class TutorService:
    def __init__(self):
        # TODO: Initialize SessionManager to handle session data
        self.session_manager = ________
        self.system_prompt = self.load_system_prompt('data/system_prompt.txt')

    def load_system_prompt(self, file_path: str) -> str:
        """Load the system prompt from file."""
        try:
            with open(file_path, 'r') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading system prompt: {e}")
            return "You are a helpful tutor."

    # TODO: Define the create_session method
    # - Parameters: student_id
    # - Generate a unique session ID using uuid
    # - Use session_manager to create a session with student_id, session_id, and system_prompt
    # - Return the session_id


from services.tutor_service import TutorService

# TODO: Initialize a TutorService instance

# TODO: Define a variable with a sample student identifier, e.g., "student123"

# TODO: Call the create_session method with the student ID and store the session ID

# TODO: Print the session ID

```

Here’s the **implementation for `create_session`** within `TutorService` plus the code to test session creation.
Just follow the comments and replace the `TODO` lines as shown below!

---

```python
import uuid
from models.session import SessionManager


class TutorService:
    def __init__(self):
        # Initialize SessionManager to handle session data
        self.session_manager = SessionManager()
        self.system_prompt = self.load_system_prompt('data/system_prompt.txt')

    def load_system_prompt(self, file_path: str) -> str:
        """Load the system prompt from file."""
        try:
            with open(file_path, 'r') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading system prompt: {e}")
            return "You are a helpful tutor."

    def create_session(self, student_id: str) -> str:
        """Create a new tutoring session."""
        session_id = str(uuid.uuid4())
        self.session_manager.create_session(student_id, session_id, self.system_prompt)
        return session_id
```

---

### **Testing Session Creation**

```python
from services.tutor_service import TutorService

# Initialize a TutorService instance
tutor_service = TutorService()

# Define a variable with a sample student identifier
student_id = "student123"

# Call the create_session method with the student ID and store the session ID
session_id = tutor_service.create_session(student_id)

# Print the session ID
print(f"Session created with ID: {session_id}")
```

---

**Expected Output Example:**

```
Session created with ID: 123e4567-e89b-12d3-a456-426614174000
```

You can now **create and verify tutoring sessions**!
Let me know if you want to see how to check the session data or proceed with adding messages.


## Implementing Session Creation in the Tutor Service

You've done well setting up the TutorService class and creating tutoring sessions. Now, let's begin implementing the process_query method to handle student queries. For now, we will focus on storing the student's query without getting a response from the AI. Here's what you need to do:

Implement the process_query method within the TutorService class.

Check if the tutoring session exists using the get_session method from SessionManager.
If the session doesn't exist, raise a ValueError with the message "Session not found".
Add the student's query to the session history using the add_message method from SessionManager.
Return a string indicating the query was processed, e.g., "Query processed".
Test the process_query method by simulating a student query, ensuring the session history is updated correctly, and handle any exceptions by printing them if the tutoring session does not exist.

This task will ensure that your tutor service can effectively handle student queries, setting the stage for integrating AI responses in the future.

```python
import uuid
from models.session import SessionManager


class TutorService:
    def __init__(self):
        self.session_manager = SessionManager()
        self.system_prompt = self.load_system_prompt('data/system_prompt.txt')

    def load_system_prompt(self, file_path: str) -> str:
        """Load the system prompt from file."""
        try:
            with open(file_path, 'r') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading system prompt: {e}")
            return "You are a helpful tutor."

    def create_session(self, student_id: str) -> str:
        """Create a new tutoring session."""
        session_id = str(uuid.uuid4())
        self.session_manager.create_session(student_id, session_id, self.system_prompt)
        return session_id
        
    # TODO: Define the process_query method
    # - Parameters: student_id, session_id, query
    # - Check if the tutoring session exists using get_session
    #   - If the session doesn't exist, raise a ValueError with "Session not found"
    # - Add the student's query to the session history using add_message
    # - Return a string indicating the query was processed


from services.tutor_service import TutorService

# Initialize the tutor service
tutor_service = TutorService()

# Simulate a student ID
student_id = "student123"

# Create a new tutoring session
session_id = tutor_service.create_session(student_id)

# TODO: Simulate sending a query and handle potential exceptions

```

Absolutely! Let’s implement the `process_query` method in your `TutorService` class according to your specs, **and** add a test for it.

---

### 1. Implement `process_query` in `TutorService`

```python
class TutorService:
    # ... (existing code)

    def process_query(self, student_id: str, session_id: str, query: str) -> str:
        """Store the student's query in the session history."""
        session = self.session_manager.get_session(student_id, session_id)
        if not session:
            raise ValueError("Session not found")
        self.session_manager.add_message(student_id, session_id, "student", query)
        return "Query processed"
```

---

### 2. Test the `process_query` Method

Add the following test code after your session creation:

```python
try:
    # Simulate a student query
    query = "What is the Pythagorean theorem?"
    result = tutor_service.process_query(student_id, session_id, query)
    print(result)  # Should print "Query processed"

    # Optional: Check that the message was actually added
    conversation = tutor_service.session_manager.get_conversation(student_id, session_id)
    for msg in conversation:
        print(msg)

except ValueError as e:
    print(f"Error: {e}")
```

#### To test a non-existent session (optional):

```python
try:
    tutor_service.process_query(student_id, "invalid-session-id", "Hello?")
except ValueError as e:
    print(f"Handled exception: {e}")  # Should print "Session not found"
```

---

### **Summary**

* `process_query` checks if the session exists, raises if not, adds the query to the session, and returns a confirmation.
* The test simulates a student query and prints the result and conversation history.

Let me know if you want to move on to integrating AI responses or any other improvements!


## Implementing Query Processing for Tutor Service

Nice work on setting up the process_query method! Now, let's enhance it to generate AI responses. Your task is to add the missing code that interacts with the DeepSeek client to obtain the AI response and update the session history.

Here's what you need to do in the TutorService class:

Instantiate the DeepSeek client to interact with the AI model.
Retrieve the conversation using get_conversation from SessionManager.
Use the DeepSeek client to generate a response based on the conversation, set the temperature to 0.7 and the maximum tokens to 500 to balance creativity and coherence in responses.
Extract the AI's response from the DeepSeek client response.
Add the AI's response to the session history.
Return the AI's response.
Remember to handle exceptions during the DeepSeek client interaction by raising a RuntimeError with an appropriate message.

The main code is pre-configured, allowing you to test the integration seamlessly. Keep going; you're doing well!

```python
import uuid
from openai import OpenAI
from models.session import SessionManager


class TutorService:
    def __init__(self):
        self.session_manager = SessionManager()
        # TODO: Initialize the client to interact with the AI model
        self.client = _____
        self.system_prompt = self.load_system_prompt('data/system_prompt.txt')

    def load_system_prompt(self, file_path: str) -> str:
        """Load the system prompt from file."""
        try:
            with open(file_path, 'r') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading system prompt: {e}")
            return "You are a helpful tutor."

    def create_session(self, student_id: str) -> str:
        """Create a new tutoring session."""
        session_id = str(uuid.uuid4())
        self.session_manager.create_session(student_id, session_id, self.system_prompt)
        return session_id
        
    def process_query(self, student_id: str, session_id: str, query: str) -> str:
        """Process a student query and get AI response."""
        session = self.session_manager.get_session(student_id, session_id)
        if not session:
            raise ValueError("Session not found")

        # Add student query
        self.session_manager.add_message(student_id, session_id, "user", query)

        try:
            # TODO: Retrieve the conversation using the SessionManager
            
            # TODO: Use the DeepSeek client to generate a response based on the conversation
            # - Set the model to "deepseek-ai/deepseek-V3"
            # - Pass the conversation as the messages parameter to provide context
            # - Use a temperature of 0.7 to balance creativity and coherence in responses
            # - Limit the response length to 500 tokens
            
            # TODO: Extract the AI's response from the DeepSeek client response
            
            # TODO: Add the AI's response to the session history
            
            # TODO: Return the AI's response
            
        except Exception as e:
            # TODO: Handle exceptions and raise a RuntimeError with an appropriate message


from services.tutor_service import TutorService

# Initialize the tutor service
tutor_service = TutorService()

# Simulate a student ID
student_id = "student123"

# Create a new tutoring session
session_id = tutor_service.create_session(student_id)
print(f"Tutoring session created with ID: {session_id}")

# Simulate sending a query
student_query = "Hello, can you help me understand calculus?"
print(f"Student Query: {student_query}")

try:
    ai_response = tutor_service.process_query(student_id, session_id, student_query)
    print(f"AI Response: {ai_response}")
except Exception as e:
    print(f"Error: {e}")

```

Here’s how you would **enhance the `process_query` method** to generate an AI response using DeepSeek (via the OpenAI-compatible client):

---

```python
import uuid
from openai import OpenAI
from models.session import SessionManager

class TutorService:
    def __init__(self):
        self.session_manager = SessionManager()
        # Initialize the OpenAI client (DeepSeek is OpenAI-compatible)
        self.client = OpenAI()
        self.system_prompt = self.load_system_prompt('data/system_prompt.txt')

    def load_system_prompt(self, file_path: str) -> str:
        try:
            with open(file_path, 'r') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading system prompt: {e}")
            return "You are a helpful tutor."

    def create_session(self, student_id: str) -> str:
        session_id = str(uuid.uuid4())
        self.session_manager.create_session(student_id, session_id, self.system_prompt)
        return session_id

    def process_query(self, student_id: str, session_id: str, query: str) -> str:
        """Process a student query and get AI response."""
        session = self.session_manager.get_session(student_id, session_id)
        if not session:
            raise ValueError("Session not found")

        # Add student query to the session history
        self.session_manager.add_message(student_id, session_id, "user", query)

        try:
            # Retrieve the full conversation as a list of messages
            conversation = self.session_manager.get_conversation(student_id, session_id)
            
            # Interact with the DeepSeek (OpenAI-compatible) client
            response = self.client.chat.completions.create(
                model="deepseek-ai/deepseek-v3",
                messages=conversation,
                temperature=0.7,
                max_tokens=500,
            )

            # Extract the AI's response content
            ai_content = response.choices[0].message.content

            # Add the AI response to the session history
            self.session_manager.add_message(student_id, session_id, "assistant", ai_content)

            # Return the AI's response
            return ai_content

        except Exception as e:
            raise RuntimeError(f"Failed to generate AI response: {e}")
```

---

### **How it Works:**

* **Initialize client:** `self.client = OpenAI()`
* **Retrieve conversation:** `conversation = self.session_manager.get_conversation(...)`
* **Generate AI response:** Use the `.chat.completions.create` method.
* **Add response to history:** Store with `"assistant"` role.
* **Return AI's response.**

---

### **Your main code stays the same for testing.**

Let me know if you want to see the expected output or if you need more test examples!


## Implement AI Response Generation with DeepSeek in TutorService

Nice work on setting up the process_query method! Now, let's enhance it to generate AI responses. Your task is to add the missing code that interacts with the DeepSeek client to obtain the AI response and update the session history.

Here's what you need to do in the TutorService class:

Instantiate the DeepSeek client to interact with the AI model.
Retrieve the conversation using get_conversation from SessionManager.
Use the DeepSeek client to generate a response based on the conversation, set the temperature to 0.7 and the maximum tokens to 500 to balance creativity and coherence in responses.
Extract the AI's response from the DeepSeek client response.
Add the AI's response to the session history.
Return the AI's response.
Remember to handle exceptions during the DeepSeek client interaction by raising a RuntimeError with an appropriate message.

The main code is pre-configured, allowing you to test the integration seamlessly. Keep going; you're doing well!

```python
import uuid
from openai import OpenAI
from models.session import SessionManager


class TutorService:
    def __init__(self):
        self.session_manager = SessionManager()
        # TODO: Initialize the client to interact with the AI model
        self.client = _____
        self.system_prompt = self.load_system_prompt('data/system_prompt.txt')

    def load_system_prompt(self, file_path: str) -> str:
        """Load the system prompt from file."""
        try:
            with open(file_path, 'r') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading system prompt: {e}")
            return "You are a helpful tutor."

    def create_session(self, student_id: str) -> str:
        """Create a new tutoring session."""
        session_id = str(uuid.uuid4())
        self.session_manager.create_session(student_id, session_id, self.system_prompt)
        return session_id
        
    def process_query(self, student_id: str, session_id: str, query: str) -> str:
        """Process a student query and get AI response."""
        session = self.session_manager.get_session(student_id, session_id)
        if not session:
            raise ValueError("Session not found")

        # Add student query
        self.session_manager.add_message(student_id, session_id, "user", query)

        try:
            # TODO: Retrieve the conversation using the SessionManager
            
            # TODO: Use the DeepSeek client to generate a response based on the conversation
            # - Set the model to "deepseek-ai/deepseek-V3"
            # - Pass the conversation as the messages parameter to provide context
            # - Use a temperature of 0.7 to balance creativity and coherence in responses
            # - Limit the response length to 500 tokens
            
            # TODO: Extract the AI's response from the DeepSeek client response
            
            # TODO: Add the AI's response to the session history
            
            # TODO: Return the AI's response
            
        except Exception as e:
            # TODO: Handle exceptions and raise a RuntimeError with an appropriate message


from services.tutor_service import TutorService

# Initialize the tutor service
tutor_service = TutorService()

# Simulate a student ID
student_id = "student123"

# Create a new tutoring session
session_id = tutor_service.create_session(student_id)
print(f"Tutoring session created with ID: {session_id}")

# Simulate sending a query
student_query = "Hello, can you help me understand calculus?"
print(f"Student Query: {student_query}")

try:
    ai_response = tutor_service.process_query(student_id, session_id, student_query)
    print(f"AI Response: {ai_response}")
except Exception as e:
    print(f"Error: {e}")

```

Here’s how you would **enhance the `process_query` method** to generate an AI response using DeepSeek (via the OpenAI-compatible client):

---

```python
import uuid
from openai import OpenAI
from models.session import SessionManager

class TutorService:
    def __init__(self):
        self.session_manager = SessionManager()
        # Initialize the OpenAI client (DeepSeek is OpenAI-compatible)
        self.client = OpenAI()
        self.system_prompt = self.load_system_prompt('data/system_prompt.txt')

    def load_system_prompt(self, file_path: str) -> str:
        try:
            with open(file_path, 'r') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading system prompt: {e}")
            return "You are a helpful tutor."

    def create_session(self, student_id: str) -> str:
        session_id = str(uuid.uuid4())
        self.session_manager.create_session(student_id, session_id, self.system_prompt)
        return session_id

    def process_query(self, student_id: str, session_id: str, query: str) -> str:
        """Process a student query and get AI response."""
        session = self.session_manager.get_session(student_id, session_id)
        if not session:
            raise ValueError("Session not found")

        # Add student query to the session history
        self.session_manager.add_message(student_id, session_id, "user", query)

        try:
            # Retrieve the full conversation as a list of messages
            conversation = self.session_manager.get_conversation(student_id, session_id)
            
            # Interact with the DeepSeek (OpenAI-compatible) client
            response = self.client.chat.completions.create(
                model="deepseek-ai/deepseek-v3",
                messages=conversation,
                temperature=0.7,
                max_tokens=500,
            )

            # Extract the AI's response content
            ai_content = response.choices[0].message.content

            # Add the AI response to the session history
            self.session_manager.add_message(student_id, session_id, "assistant", ai_content)

            # Return the AI's response
            return ai_content

        except Exception as e:
            raise RuntimeError(f"Failed to generate AI response: {e}")
```

---

### **How it Works:**

* **Initialize client:** `self.client = OpenAI()`
* **Retrieve conversation:** `conversation = self.session_manager.get_conversation(...)`
* **Generate AI response:** Use the `.chat.completions.create` method.
* **Add response to history:** Store with `"assistant"` role.
* **Return AI's response.**

---

### **Your main code stays the same for testing.**

Let me know if you want to see the expected output or if you need more test examples!


## Testing Context Retention in AI Tutor Conversations

You've successfully set up the tutor service and processed the initial query. Now, let's enhance the AI's capability by testing its ability to retain context across multiple interactions. This will ensure the AI can handle multi-turn conversations effectively.

Here's what you need to do:

After sending the initial query, define a follow-up question that requires the AI to recall the context of the previous query. For example, you might ask: "Is there another way?" Print this follow-up question to the console.

Use the process_query method to send this follow-up question to the AI.

Print the AI's response to the follow-up question to verify its ability to maintain context across multiple queries.

This task will help you verify the AI's ability to maintain context across multiple queries. Keep going; you're doing well!

```python
from services.tutor_service import TutorService

# Initialize the tutor service
tutor_service = TutorService()

# Simulate a student ID
student_id = "student123"

# Create a new tutoring session
session_id = tutor_service.create_session(student_id)
print(f"Tutoring session created with ID: {session_id}")

# Simulate sending a query
student_query = "Hello, can you help me understand calculus?"
print(f"Student Query: {student_query}")

try:
    ai_response = tutor_service.process_query(student_id, session_id, student_query)
    print(f"AI Response: {ai_response}")
except Exception as e:
    print(f"Error: {e}")

# TODO: Define and print a follow-up question
# TODO: Send the follow-up question to test context retention
# TODO: Print the AI's response to the follow-up question


import uuid
from openai import OpenAI
from models.session import SessionManager


class TutorService:
    def __init__(self):
        self.session_manager = SessionManager()
        self.client = OpenAI()
        self.system_prompt = self.load_system_prompt('data/system_prompt.txt')

    def load_system_prompt(self, file_path: str) -> str:
        """Load the system prompt from file."""
        try:
            with open(file_path, 'r') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading system prompt: {e}")
            return "You are a helpful tutor."

    def create_session(self, student_id: str) -> str:
        """Create a new tutoring session."""
        session_id = str(uuid.uuid4())
        self.session_manager.create_session(student_id, session_id, self.system_prompt)
        return session_id
        
    def process_query(self, student_id: str, session_id: str, query: str) -> str:
        """Process a student query and get AI response."""
        session = self.session_manager.get_session(student_id, session_id)
        if not session:
            raise ValueError("Session not found")

        # Add student query
        self.session_manager.add_message(student_id, session_id, "user", query)

        try:
            # Get AI response
            conversation = self.session_manager.get_conversation(student_id, session_id)
            
            response = self.client.chat.completions.create(
                model="deepseek-ai/deepseek-V3",
                messages=conversation,
                temperature=0.7,
                max_tokens=500
            )
            
            ai_message = response.choices[0].message.content
            
            # Add AI response to session history
            self.session_manager.add_message(student_id, session_id, "assistant", ai_message)
            
            return ai_message
            
        except Exception as e:
            raise RuntimeError(f"Error getting AI response: {str(e)}")


```

Perfect! You’re almost there. To **test the AI’s context retention**, just follow these steps after your initial query/response block.

Add this directly after your existing `try/except`:

```python
# Define a follow-up question that depends on previous context
follow_up_query = "Is there another way?"
print(f"Follow-up Question: {follow_up_query}")

try:
    # Send the follow-up question to test context retention
    follow_up_response = tutor_service.process_query(student_id, session_id, follow_up_query)
    print(f"AI Response to Follow-up: {follow_up_response}")
except Exception as e:
    print(f"Error: {e}")
```

---

**Full Main Code Example (append after your current code):**

```python
# ... previous code

# Define a follow-up question that requires the AI to recall context
follow_up_query = "Is there another way?"
print(f"Follow-up Question: {follow_up_query}")

try:
    # Send the follow-up question
    follow_up_response = tutor_service.process_query(student_id, session_id, follow_up_query)
    print(f"AI Response to Follow-up: {follow_up_response}")
except Exception as e:
    print(f"Error: {e}")
```

---

### **What does this do?**

* **Step 1:** Sends a follow-up question that is only meaningful if the AI remembers the prior topic (calculus).
* **Step 2:** Prints the follow-up and the AI's response, verifying context retention.

---

If the AI response **refers to calculus or methods** related to your original query, then your multi-turn memory works!
Ready for more? Let me know if you want to test with even deeper follow-ups or need to review the session’s message history for debugging.


## Creating Multiple Tutoring Sessions for a Student

You've made great progress in setting up the TutorService class and processing student queries. Now, let's focus on creating multiple tutoring sessions for the same student to ensure that each session has a cleared context.

Here's what you need to do:

Create a Second Tutoring Session: Use the create_session method to create a new tutoring session for the same student. This will ensure that the student can have multiple independent learning conversations.

Process a Query in the Second Session: Simulate sending a query in the newly created tutoring session using the process_query method. This will test that the tutoring session is functioning correctly and independently from the first session.

This task will guide you through the process of creating and managing tutoring sessions, a fundamental aspect of building a personal tutor service. Keep up the momentum!

```python
from services.tutor_service import TutorService

# Initialize the tutor service
tutor_service = TutorService()

# Simulate a student ID
student_id = "student123"

# Create a new tutoring session
session_id = tutor_service.create_session(student_id)
print(f"Tutoring session created with ID: {session_id}")

# Simulate sending a query
student_query = "In what year Rome burned down?"
print(f"Student Query: {student_query}")

try:
    ai_response = tutor_service.process_query(student_id, session_id, student_query)
    print(f"AI Response: {ai_response}")
except Exception as e:
    print(f"Error: {e}")

# TODO: Create a second tutoring session for the same student and process a new query
```

Absolutely! Here’s how you can **create a second tutoring session for the same student** and then **process a new query** in that session. This ensures each session is isolated, with its own context.

Let’s expand your script with the missing steps:

```python
from services.tutor_service import TutorService

# Initialize the tutor service
tutor_service = TutorService()

# Simulate a student ID
student_id = "student123"

# --- First Session ---
# Create a new tutoring session
session_id_1 = tutor_service.create_session(student_id)
print(f"Tutoring session 1 created with ID: {session_id_1}")

# Simulate sending a query in the first session
student_query_1 = "In what year Rome burned down?"
print(f"Student Query 1: {student_query_1}")

try:
    ai_response_1 = tutor_service.process_query(student_id, session_id_1, student_query_1)
    print(f"AI Response 1: {ai_response_1}")
except Exception as e:
    print(f"Error in session 1: {e}")

# --- Second Session ---
# Create a second tutoring session for the same student
session_id_2 = tutor_service.create_session(student_id)
print(f"Tutoring session 2 created with ID: {session_id_2}")

# Simulate sending a query in the second session
student_query_2 = "Explain the Pythagorean theorem."
print(f"Student Query 2: {student_query_2}")

try:
    ai_response_2 = tutor_service.process_query(student_id, session_id_2, student_query_2)
    print(f"AI Response 2: {ai_response_2}")
except Exception as e:
    print(f"Error in session 2: {e}")

# (Optional) You can verify that each session maintains its own conversation context by
# checking the session histories if your TutorService exposes such functionality.
```

---

### **What’s Happening?**

* **Two sessions** are created for the same student (`student123`).
* Each session gets its **own context**, meaning questions and conversation are isolated to that session.
* Queries in each session do **not affect each other**.

---

**Tip:**
If you want to check that the context is indeed isolated, you could print or inspect the session histories if your implementation of `TutorService` or `SessionManager` allows it. If not, you can add a method like `get_conversation(student_id, session_id)` to inspect the stored messages per session.

---

Let me know if you want a sample implementation of such a method, or have any other questions!
